In [1]:
import os
import git
from pathlib import Path

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

In [7]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
from reporting import *
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")
save_path_with_hull = Path(os.path.join(ROOT_DIR, 'publication', 'paper', 'CSVs', 'final_results_with_hull.pickle'))
main_df = pd.read_pickle(save_path_with_hull)
RERUN = False
np.random.seed(0)

In [8]:
main_df = main_df.fillna("None", inplace=False)

In [9]:
main_df

,group,dataset,subset,transform,orientation,channel,dataset_type,obs_var,var_lower,var_upper,...,kstest_stat_t,kstest_pval_t,kstest_pval_gengamma,github_plot,total_samples,beat_all_priors,best_prior,failure_category,failure_type,which_ones
0,2,syntheticMRI3D,full,wavelet,ada,None,medical,None,None,None,...,None,None,None,https://github.com/yashdave003/hierarchical-ba...,0.0,0,nan,trivial_failure,low samples,None
1,3,syntheticMRI3D,full,wavelet,ada,None,medical,1917588.4,1678674.1,2165048.5,...,0.057539,0.045912,0.081459,https://github.com/yashdave003/hierarchical-ba...,564.0,0,Gaussian,trivial_failure,multimodal,None
2,4,syntheticMRI3D,full,wavelet,ada,None,medical,814282.0,777582.6,851497.5,...,0.017056,0.000004,0.000005,https://github.com/yashdave003/hierarchical-ba...,22538.0,0,Laplace,interesting_failure,double inflection,None
3,5,syntheticMRI3D,full,wavelet,ada,None,medical,249365.95,231914.97,267766.94,...,0.00445,0.000016,0.001396,https://github.com/yashdave003/hierarchical-ba...,296776.0,1,GenGamma,practically_pass,pass,None
4,6,syntheticMRI3D,full,wavelet,ada,None,medical,43869.16,40573.156,47332.902,...,0.011601,0.0,0.0,https://github.com/yashdave003/hierarchical-ba...,2880648.0,1,GenGamma,practically_pass,pass,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,6,standardTesting,full,wavelet,vertical,gray,natural,58.38707,50.69825,66.48225,...,0.042903,0.000402,0.863448,https://github.com/yashdave003/hierarchical-ba...,2304.0,1,GenGamma,borderline,low samples,None
1151,7,standardTesting,full,wavelet,vertical,gray,natural,10.897633,10.028848,11.798168,...,0.053283,0.0,0.03391,https://github.com/yashdave003/hierarchical-ba...,9216.0,1,GenGamma,practically_pass,pass,None
1152,8,standardTesting,full,wavelet,vertical,gray,natural,2.093454,1.924215,2.268933,...,0.049858,0.0,0.032072,https://github.com/yashdave003/hierarchical-ba...,36864.0,1,GenGamma,practically_pass,pass,None
1153,9,standardTesting,full,wavelet,vertical,gray,natural,0.375532,0.342325,0.410297,...,0.047751,0.0,0.0,https://github.com/yashdave003/hierarchical-ba...,147456.0,1,GenGamma,practically_pass,pass,None


In [10]:
granularity = ['dataset', 'transform']
temp = main_df.copy()[(main_df['dataset'] != 'standardTesting')] #  & (main_df['transform'] != 'learned')
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1)
summary_df

TypeError: agg function failed [how->mean,dtype->object]

In [6]:
granularity = ['dataset', 'transform']
temp = main_df.copy()[(main_df['dataset'] != 'standardTesting')]# & (main_df['transform'] != 'learned')]
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
temp['borderline'] = (temp['failure_category'] == 'borderline')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1).sort_values('transform')
summary_df.reset_index()#.drop('dataset', axis=1)

,dataset,transform,actually_pass,practically_pass,intersect_roi
0,agriVision,fourier,2.3,93.2,70.5
1,pastis,fourier,90.9,9.1,100.0
2,spaceNet,fourier,0.0,100.0,50.0
3,agriVision,learned,0.0,78.3,53.3
4,coco,learned,0.0,57.9,33.3
5,pastis,learned,0.0,85.7,74.3
6,segmentAnything,learned,0.0,73.4,50.0
7,spaceNet,learned,0.0,42.9,46.4
8,agriVision,wavelet,13.9,86.1,73.6
9,coco,wavelet,25.0,26.6,27.1


In [7]:
save_path = Path(os.path.join(ROOT_DIR, 'publication', 'poster', 'CSVs', 'summary_table.csv'))
summary_df.to_csv(save_path) #.drop('learned')

In [16]:
summary_df = summary_df.reset_index()
summary_df

,index,dataset,transform,actually_pass,practically_pass,intersect_roi
0,0,agriVision,fourier,2.3,93.2,70.5
1,1,pastis,fourier,90.9,9.1,100.0
2,2,spaceNet,fourier,0.0,100.0,50.0
3,3,agriVision,learned,0.0,78.3,53.3
4,4,coco,learned,0.0,57.9,33.3
5,5,pastis,learned,0.0,85.7,74.3
6,6,segmentAnything,learned,0.0,73.4,50.0
7,7,spaceNet,learned,0.0,42.9,46.4
8,8,agriVision,wavelet,13.9,86.1,73.6
9,9,coco,wavelet,25.0,26.6,27.1


In [9]:
(temp[['actually_pass','practically_pass','intersect_roi', 'borderline']].mean(numeric_only=True)*100).round(1)

actually_pass       15.5
practically_pass    56.3
intersect_roi       50.2
borderline           4.8
dtype: float64

In [17]:
(temp.groupby(['transform'])[['kstest_stat_best', 'actually_pass','practically_pass','intersect_roi', 'total_samples']].median(numeric_only=True)*1)#.round(1)

,kstest_stat_best,actually_pass,practically_pass,intersect_roi,total_samples
transform,,,,,
fourier,0.003361,0.0,1.0,1.0,552780.0
learned,0.007980,0.0,1.0,0.0,480000000.0
wavelet,0.006935,0.0,1.0,0.0,576000.0


In [18]:
(temp.groupby(['dataset', 'transform'])[['kstest_stat_best', 'actually_pass','practically_pass','intersect_roi', 'total_samples']].median(numeric_only=True)*1)#.round(1)

kstest_stat_best  actually_pass  practically_pass  \
dataset         transform                                                      
agriVision      fourier            0.004708            0.0               1.0   
                learned            0.005417            0.0               1.0   
                wavelet            0.005020            0.0               1.0   
coco            learned            0.009714            0.0               1.0   
                wavelet            0.011939            0.0               0.0   
pastis          fourier            0.001526            1.0               0.0   
                learned            0.006469            0.0               1.0   
                wavelet            0.004502            1.0               0.0   
segmentAnything learned            0.004959            0.0               1.0   
                wavelet            0.004032            0.0               1.0   
spaceNet        fourier            0.005150            0.0               1.0   
                learned            0.012170            0.0               0.0   
                wavelet            0.011693            0.0               0.0   
syntheticMRI2D  wavelet            0.005400            0.0               1.0   
syntheticMRI3D  wavelet            0.004573            0.0               1.0   

                           intersect_roi  total_samples  
dataset         transform                                
agriVision      fourier              1.0      2574000.0  
                learned              1.0    786432000.0  
                wavelet              1.0      1728000.0  
coco            learned              0.0    196608000.0  
                wavelet              0.0       283584.0  
pastis          fourier              1.0       257580.0  
                learned              1.0       491520.0  
                wavelet              1.0       152640.0  
segmentAnything learned              0.5    786432000.0  
                wavelet              0.0      1810432.0  
spaceNet        fourier              0.5      1816134.0  
                learned              0.0    480000000.0  
                wavelet              0.0      1637581.5  
syntheticMRI2D  wavelet              0.0      1670460.0  
syntheticMRI3D  wavelet              1.0       296776.0